In [ ]:
import os
os.environ["TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL"] = "1"

In [ ]:
from datasets import load_dataset

datasets = load_dataset("0xDing/wikipedia-cn-20230720-filtered", split="train")

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-macbert-base").to("cuda")

In [ ]:
def process_function(examples, tokenizer=tokenizer):
    return tokenizer(examples["completion"], max_length=384, truncation=True)

In [ ]:
tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets.column_names)

In [ ]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)

In [ ]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir="./masked_lm",
    per_device_train_batch_size=32,
    # gradient_accumulation_steps=2,
    logging_steps=10,
    num_train_epochs=1,
    report_to=["tensorboard"],
)

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_datasets["train"].select(range(int(len(tokenized_datasets["train"]) / 5))),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15),
)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline

pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [ ]:
pipe("西安交通[MASK][MASK]博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆")

In [ ]:
pipe("第一把手枪局输了的情况下第二把应该[MASK][MASK]")